## Synthesis Summary Explaination
### Headers
- Benchmark Name
- Total Depenedencies
- Dependencies Ratio
- Realizable (If one of the tools claims for realizability)
- (Tool name) Duration
- (Tool name) Status (ERROR | COMPLETED | TIMEOUT)

### Tools
 - Deps Synthesis (Synthesis 2 strategies: dependents, indepenents)
 - Spot Hijack (Don't find deps + don't eject)
 - Indep Synthesis (Synhesis a strategy only for independents (eject only))
 - Spot SD
 - Spot LAR
 - Spot PS
 - Strix

### Sanity Check
- Realizability is similar in all tools

In [13]:
import pandas as pd
from matplotlib import pyplot as plt
from glob import glob
import os.path
from pathlib import Path
import json
import re

## Loading Deps Synthesiser Results

In [ ]:
def count_dependent_vars(testedVars):
    return 

In [28]:
# TODO: all paths extract to global const file
deps_synt_hoa_files = glob("../assets/aiger-synthesis/bfss-with-deps/*.hoa")

def deps_synt_analysis(benchmark_name):
    hoa_file_path = "../assets/aiger-synthesis/bfss-with-deps/{}.hoa".format(benchmark_name)
    json_file_path = "../assets/aiger-synthesis/bfss-with-deps/{}.json".format(benchmark_name)
    hoa_file = Path(hoa_file_path).read_text()
    
    # If JSON file doesn't exist, it we had error in synthesis
    if not os.path.exists(json_file_path):
        status = 'error'
        error = re.findall('Runtime error:(.*)', hoa_file)
        
        return {
            'name': benchmark_name,
            'status': 'error',
            'error': error
        }
    
    measures = json.loads(Path(json_file_path).read_text())
    
    duration = measures['total_time']
    status = 'completed' if measures['is_completed'] == True  else 'timeout'
    if status != 'completed':
        return {
            'name': benchmark_name,
            'duration': duration,
            'status': status,
        }
    
    total_dependents = sum((1 for v in measures['tested_variables'] if v['is_dependent']))
    total_outputs = len(measures['output_vars'])
    dependents_ratio = total_dependents / total_outputs
    
    return {
        'name': benchmark_name,
        'duration': duration,
        'status': status,
        'dependents_ratio': dependents_ratio,
        'total_dependents': total_dependents,
    }

deps_synt_df = pd.DataFrame([
    deps_synt_analysis(Path(benchmark_path).stem)
    for benchmark_path in deps_synt_hoa_files
])


deps_synt_df
# deps_synt_df[deps_synt_df['status'] == 'timeout']

,name,duration,status,dependents_ratio,total_dependents,error
0,TwoCountersInRangeA4,210.0,completed,0.375000,3.0,NaN
1,ltl2dba05,6.0,completed,0.000000,0.0,NaN
2,ltl2dba11,2.0,completed,0.000000,0.0,NaN
3,TwoCountersDisButA6,12926.0,completed,0.307692,4.0,NaN
4,TorcsSteeringSimple,5.0,completed,0.333333,1.0,NaN
...,...,...,...,...,...,...
202,OneCounterGuiA1,22.0,completed,0.333333,3.0,NaN
203,EscalatorSmart,8.0,completed,0.333333,1.0,NaN
204,OneCounterGuiA0,41.0,completed,0.333333,3.0,NaN
205,ltl2dba20,9.0,completed,0.000000,0.0,NaN
